In [1]:
import pandas as pd
import numpy as np
import metric as me
import swco
import k_mers_metric as kmm

Read the k_mers

In [2]:
# Just the ones with prime k values
#k_mers = me.read_k_mers_prime('../Data/Intermediate/k_mers/Scaffold')

# Version to read all of them: me.read_k_mers(path), not only for k prime numbers

# aprox. 

101 :  101_mers.csv
103 :  103_mers.csv
107 :  107_mers.csv
109 :  109_mers.csv
113 :  113_mers.csv
11 :  11_mers.csv
127 :  127_mers.csv
131 :  131_mers.csv
137 :  137_mers.csv
139 :  139_mers.csv
13 :  13_mers.csv
149 :  149_mers.csv
151 :  151_mers.csv
157 :  157_mers.csv
163 :  163_mers.csv
167 :  167_mers.csv
173 :  173_mers.csv
179 :  179_mers.csv
17 :  17_mers.csv
181 :  181_mers.csv
191 :  191_mers.csv
193 :  193_mers.csv
197 :  197_mers.csv
199 :  199_mers.csv
19 :  19_mers.csv
211 :  211_mers.csv
223 :  223_mers.csv
227 :  227_mers.csv
229 :  229_mers.csv
233 :  233_mers.csv
239 :  239_mers.csv
23 :  23_mers.csv
241 :  241_mers.csv
29 :  29_mers.csv
2 :  2_mers.csv
31 :  31_mers.csv
37 :  37_mers.csv
3 :  3_mers.csv
41 :  41_mers.csv
43 :  43_mers.csv
47 :  47_mers.csv
53 :  53_mers.csv
59 :  59_mers.csv
5 :  5_mers.csv
61 :  61_mers.csv
67 :  67_mers.csv
71 :  71_mers.csv
73 :  73_mers.csv
79 :  79_mers.csv
7 :  7_mers.csv
83 :  83_mers.csv
89 :  89_mers.csv
97 :  97_mers.cs

In [9]:
df_k_mers = pd.read_feather('../Data/Intermediate/k_mers/processed/df_k_mers_relaxed.feather')

In [4]:
'''# Let's read the new k-mers from GFF files
import read_k_mers as rkm

# Path to the folder with the processed files
path = '../Data/Intermediate/k_mers/processed/'
data_list = rkm.process_folder(path)

# Convert the list of dictionaries into a single DataFrame, including the species and k columns
df_list = pd.concat([d['data'].assign(species_gcf_id=d['species_gcf_id'], k=d['k']) for d in data_list], ignore_index=True)
#df_list = pd.concat([d['data'] for d in data_list], ignore_index=True)
df_list = df_list[['species_gcf_id', 'k', 'k_mer', 'replicon_accession']].rename(columns={'replicon_accession': 'scaffold'})

# Create a column in the dataframe with the species name
file_species = rkm.map_species_to_file()
df_list['species'] = df_list['species_gcf_id'].map(file_species)'''

"# Let's read the new k-mers from GFF files\nimport read_k_mers as rkm\n\n# Path to the folder with the processed files\npath = '../Data/Intermediate/k_mers/processed/'\ndata_list = rkm.process_folder(path)\n\n# Convert the list of dictionaries into a single DataFrame, including the species and k columns\ndf_list = pd.concat([d['data'].assign(species_gcf_id=d['species_gcf_id'], k=d['k']) for d in data_list], ignore_index=True)\n#df_list = pd.concat([d['data'] for d in data_list], ignore_index=True)\ndf_list = df_list[['species_gcf_id', 'k', 'k_mer', 'replicon_accession']].rename(columns={'replicon_accession': 'scaffold'})\n\n# Create a column in the dataframe with the species name\nfile_species = rkm.map_species_to_file()\ndf_list['species'] = df_list['species_gcf_id'].map(file_species)"

Read the species and their lengths

In [5]:
df_species = pd.read_csv('../Data/Intermediate/species_length.csv')        #sorted(df_species['Specie'].unique())
df_species

,Specie,Number of genes
0,Aadvark,14841
1,Alligator M,13596
2,Alligator S,13140
3,Anolis,12650
4,Chelonia,12293
5,Chrysemys,11218
6,Croco,12138
7,Danio,25221
8,Devil,14059
9,Dog,60


In [6]:
'''
When I have the new GFF files, compute the length of the scaffolds and the species
Then, we can create the measure matrix with this code

import k_mers_metric as kmm

# Given a specific scaffold, know how many k-mers shared with the rest of scaffolds
level = 'species'

k_mers_counter = pd.read_feather('../Data/Intermediate/k_mers/count_k_mers_' + level + '.feather')

try:
    df_length = pd.read_csv('../Data/Intermediate/' + level + '_length.csv')
    if level == 'scaffold':
        df_length = (df_length[['Specie', 'Replicon Accession', 'Number of genes']]
                        .rename(columns = {'Specie': 'species', 'Replicon Accession': 'scaffold', 'Number of genes': 'length'}))
    elif level == 'species':
        df_length = (df_length[['Specie', 'Number of genes']]
                        .rename(columns = {'Specie': 'species', 'Number of genes': 'length'}))
except:
    print('Please, check the columns of the file ' + level + '_length.csv')

measure = pd.DataFrame()

for species in df_species['Specie']:
    print(species)
    species = kmm.k_mers_to_metric(species, level, k_mers_counter, df_length, df_k_mers)
    species['species1'] = origin
    species.rename(columns={'species2': 'species'}, inplace=True)
    
    measure = pd.concat([measure, species])
'''


"\nWhen I have the new GFF files, compute the length of the scaffolds and the species\nThen, we can create the measure matrix with this code\n\nimport k_mers_metric as kmm\n\n# Given a specific scaffold, know how many k-mers shared with the rest of scaffolds\nlevel = 'species'\n\nk_mers_counter = pd.read_feather('../Data/Intermediate/k_mers/count_k_mers_' + level + '.feather')\n\ntry:\n    df_length = pd.read_csv('../Data/Intermediate/' + level + '_length.csv')\n    if level == 'scaffold':\n        df_length = (df_length[['Specie', 'Replicon Accession', 'Number of genes']]\n                        .rename(columns = {'Specie': 'species', 'Replicon Accession': 'scaffold', 'Number of genes': 'length'}))\n    elif level == 'species':\n        df_length = (df_length[['Specie', 'Number of genes']]\n                        .rename(columns = {'Specie': 'species', 'Number of genes': 'length'}))\nexcept:\n    print('Please, check the columns of the file ' + level + '_length.csv')\n\nmeasure = pd

In [8]:
df_species

,Specie,Number of genes
0,Aadvark,14841
1,Alligator M,13596
2,Alligator S,13140
3,Anolis,12650
4,Chelonia,12293
5,Chrysemys,11218
6,Croco,12138
7,Danio,25221
8,Devil,14059
9,Dog,60


In [7]:
metric = []

for index_sp1 in range(len(df_species)-1):
    sp1, len_sp1 = df_species[index_sp1]

    for index_sp2 in range(index_sp1+1, len(df_species)):
        sp2, len_sp2 = df_species[index_sp2]

        current = { 
                    'Specie1': sp1,
                    'Specie2': sp2,
                    'Ratio': me.metric_comp(sp1, sp2, k_mers, len_sp1, len_sp2)
                }
        print(current)
        
        metric.append(current)
metric = pd.DataFrame(metric)

#148 min (with iterruption)

KeyError: 0

In [ ]:
me.metric_to_matrix(metric).to_csv('../Data/Intermediate/metric_matrix_relaxed_table.csv', index=False)

AttributeError: module 'metric' has no attribute 'metric_to_matrix'

In [ ]:
# Transform it into a distance
#me.metric_to_matrix(metric[['Specie1', 'Specie2', 'Distance_1']]).to_csv('../Data/Results/Metric/Distance/distance_1.csv', index=False)
#me.metric_to_matrix(metric[['Specie1', 'Specie2', 'Distance_1/2']]).to_csv('../Data/Results/Metric/Distance/distance_12.csv', index=False)